<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Quantum_Entangled_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq tensorflow_quantum

In [ ]:
# Required imports
import numpy as np
import sympy as sp
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq

# Qubits and base entangling circuit
q0, q1 = cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)

# Symbols for classical features (2D -> two rotations)
x0, x1 = sp.symbols('x0 x1')

# Feature-encoding, entangled circuit
circuit = cirq.Circuit()
circuit.append([cirq.H(q0), cirq.CNOT(q0, q1)])
# Encode features into rotations (you can choose encodings you like)
circuit.append([cirq.rx(x0)(q0), cirq.ry(x1)(q1)])

# Readout operator(s)
readout_ops = [cirq.Z(q0)]

# Model that consumes circuits + classical features
class QuantumEntangledAI(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # ControlledPQC maps per-example feature values into the circuit symbols [x0, x1]
        self.quantum = tfq.layers.ControlledPQC(circuit, readout_ops)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, inputs):
        circ_inp, feat_inp = inputs  # [batch], [batch, 2]
        expvals = self.quantum([circ_inp, feat_inp])  # shape: [batch, 1]
        return self.dense(expvals)

# Synthetic training data
n = 1000
# Base circuits for each example (can be empty—entanglement lives in the layer’s circuit)
base_circuits = [cirq.Circuit()] * n
x_circ = tfq.convert_to_tensor(base_circuits)       # shape: [n], dtype=tf.string

# Two classical features -> feed into the symbols [x0, x1]
x_feats = np.random.uniform(-np.pi, np.pi, size=(n, 2)).astype(np.float32)

# Binary labels
y = np.random.randint(0, 2, size=(n,)).astype(np.int32)

# Build and train
model = QuantumEntangledAI()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit([x_circ, x_feats], y, epochs=10, batch_size=32, verbose=1)

In [ ]:
import numpy as np
import sympy as sp
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq

q0, q1 = cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)
theta = sp.symbols('theta')

circuit = cirq.Circuit()
circuit.append([cirq.H(q0), cirq.CNOT(q0, q1)])
circuit.append(cirq.rx(theta)(q0))
readout_ops = [cirq.Z(q0)]

# PQC takes circuits only; 'theta' becomes a trainable weight inside the layer.
inputs = tf.keras.Input(shape=(), dtype=tf.string)
expvals = tfq.layers.PQC(circuit, readout_ops)(inputs)
outputs = tf.keras.layers.Dense(2, activation='softmax')(expvals)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

n = 1000
x_circ = tfq.convert_to_tensor([cirq.Circuit()] * n)
y = np.random.randint(0, 2, size=(n,)).astype(np.int32)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_circ, y, epochs=10, batch_size=32, verbose=1)